In [1]:
import pandas as pd
import numpy as np
import os, sys, time
import yaml
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import ExtraTreesClassifier


project_path = '/mnt/home/ymm/kaggle/compete/current'
data_path = '/mnt/home/ymm/kaggle/bosch_data/bosch_complete_processed_data'
#data_path = '/mnt/home/ymm/kaggle/bosch_processed_data'

In [2]:
yaml_file = 'bosch_processed_data_dict.yml'
with open(os.path.join(project_path, yaml_file), 'r') as yml_stream:
    data_dict = yaml.load(yml_stream)
    

In [3]:
data_index = '0'
data_file = os.path.join(data_path, data_dict[data_index]['train_file'])
print 'loading data from ', data_file
train = pd.read_csv(data_file, index_col='Id')
#train = pd.read_csv(data_file, index_col='Id', nrows=1000)

loading data from  /mnt/home/ymm/kaggle/bosch_data/bosch_complete_processed_data/processed_totBins_16_bin_0_train.csv


In [4]:
sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils
#from utils.wrapped_xgboost import xgboost_classifier
from utils.models import RandomForestModel, ExtraTreeModel
from utils.validation_tools import cross_validate_model, score_MCC

In [5]:
#reload(utils.models)
#dir(utils.models)
#reload(utils.validation_tools)
#reload(utils.wrapped_xgboost)
#dir(utils.wrapped_xgboost)

In [5]:
## for other sk-learn models
dep_var_name = 'Response'
## not to separate the dep_var from data
#train_label = train[dep_var_name]
#train.drop(dep_var_name, axis=1, inplace=True)

In [26]:
'''
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import abc
import pandas as pd
import numpy as np

class BaseModel(object):
    __metaclass__ = abc.ABCMeta

    @abc.abstractmethod
    def fit(self, data):
        pass

    @abc.abstractmethod
    def predict(self, data):
        pass

    

class RandomForestModel(BaseModel):

    def __init__(self, model_params):
        super(BaseModel, self).__init__()
        self.model = RandomForestClassifier(**model_params)

    def fit(self, data, label):
        if isinstance(label, pd.Series):
            label = label.values
        self.model.fit(data, label)

    def predict(self, data):
        scores = self.model.predict_proba(data)
        return scores[:, 1]
    
    

class ExtraTreeModel(BaseModel):

    def __init__(self, model_params):
        super(BaseModel, self).__init__()
        self.model = ExtraTreesClassifier(**model_params)

    def fit(self, data, label):
        if isinstance(label, pd.Series):
            label = label.values

        self.model.fit(data, label)

    def predict(self, data):
        scores = self.model.predict_proba(data)
        return scores[:, 1]

'''

In [9]:
rf_params = {'n_estimators' : 100, 'max_depth' : 5, 'random_state' : 0}
rf_model = RandomForestModel(rf_params)
#rf_model = ExtraTreeModel(rf_params)

In [9]:
'''
tmp_model = RandomForestClassifier(**rf_params)
#tmp_model = ExtraTreesClassifier(**rf_params)

#tmp_train = train.copy()
tmp1 = train.copy()
tmp2 = tmp1[dep_var_name]
tmp1.drop(dep_var_name, axis=1, inplace=True)
tmp_model.fit(tmp1, tmp2)
res = tmp_model.predict_proba(tmp1)
'''

In [10]:
tmp_train = train.copy()
results = cross_validate_model(tmp_train, dep_var_name, rf_model, score_MCC, 2)

0.00284925652213 0.00294663176599
score shape: (22462,) mean of groud truth: 0.00284925652213
threshold value: 0.0848952207232
0.00284938337563 0.00283042118953
score shape: (22461,) mean of groud truth: 0.00284938337563
threshold value: 0.0728924114752


In [11]:
results

[0.12249977676578253, 0.12249966513372394]

In [147]:
'''
from utils.validation_tools import create_validation_index
## obtein the index for train and validation data
train_index, valid_index = create_validation_index(train, dep_var_name='Response', valid_frac=0.3)
'''

"\nfrom utils.validation_tools import create_validation_index\n## obtein the index for train and validation data\ntrain_index, valid_index = create_validation_index(train, dep_var_name='Response', valid_frac=0.3)\n"

In [51]:
import itertools

In [56]:
a = list(itertools.product([1, 2, 3], ['a', 'b'], [1]))

In [44]:
def abc(model_object, param):
    model = model_object(param)
    return model

In [46]:
a = ExtraTreeModel(rf_params)

In [47]:
a

In [62]:


def grid_search_crosvalid_model(train, dep_var_name, model_class, eval_func, param_dict, fold_num=3, result_file='grid_search_results.csv'):
    params_list = list(itertools.product(*param_dict.values()))
    df = pd.DataFrame(columns=param_dict.keys() + ['avg_score', 'score_std'])
    df.to_csv(result_file)
    row_counter = 0
    start_time = time()
    for param in params_list:
        model_params = {}
        for value, key in zip(param, param_dict.keys()):
            model_params[key] = value
            
        model = model_class(model_params)
        tmp_train = train.copy()
        results = cross_validate_model(tmp_train, dep_var_name, model, eval_func, fold_num)
        row_content = model_params.values()
        row_content.append(np.mean(results))
        row_content.append(np.std(results))
        df = pd.read_csv(result_file, index_col=0)
        df.loc[row_counter] = row_content
        df.to_csv(result_file)
        row_counter += 1
        if row_counter % 10 == 0:
            print '{} grid points are finished using {} seconds'.format(row_counter, round((time() - start_time), 0))
        
        

In [60]:
const_param_dict = {'random_state' : 0}
def list_const_params(params):
    listed_params = {}
    for key, value in params.items():
        listed_params[key] = [value]
    return listed_params
const_param_dict = list_const_params(const_param_dict)
tuning_param_dict = {'n_estimators':range(500, 5000, 500), 'max_depth':range(4, 11)}

#rf_params = {'n_estimators' : 100, 'max_depth' : 5, 'random_state' : 0}

In [ ]:
param_dict = const_param_dict.copy()
param_dict.update(tuning_param_dict)
print param_dict
list(itertools.product(*param_dict.values()))

grid_search_crosvalid_model(train, dep_var_name, ExtraTreeModel, score_MCC, param_dict)

{'n_estimators': [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500], 'random_state': [0], 'max_depth': [4, 5, 6, 7, 8, 9, 10]}
0.0028714524207 0.00285180748516
score shape: (14975,) mean of groud truth: 0.0028714524207
threshold value: 0.0610825296271
0.0028714524207 0.002895283718
score shape: (14975,) mean of groud truth: 0.0028714524207
threshold value: 0.0733789272449
0.00280504908836 0.00278911723844
score shape: (14973,) mean of groud truth: 0.00280504908836
threshold value: 0.0510899454839
0.0028714524207 0.0028090234061
score shape: (14975,) mean of groud truth: 0.0028714524207
threshold value: 0.0723569989499
0.0028714524207 0.0028381197687
score shape: (14975,) mean of groud truth: 0.0028714524207
threshold value: 0.0764744660799
0.00280504908836 0.00293338692286
score shape: (14973,) mean of groud truth: 0.00280504908836
threshold value: 0.0556635795039
0.0028714524207 0.00298178383784
score shape: (14975,) mean of groud truth: 0.0028714524207
threshold value: 0.084061037

In [ ]:
grid_search_crosvalid_model(train, dep_var_name, ExtraTreeModel, )

In [21]:
tmp

[array([[1, 2, 3],
        [1, 2, 3]]), array([[1, 1, 1],
        [2, 2, 2]])]